In [2]:
import os, sqlite3, xlrd
import pandas as pd

In [4]:
xls_file = pd.ExcelFile('query.xlsx')
table = xls_file.parse('All Samples')

In [5]:
print(table['Chr'])

        Sample                  Gene        Variant Chr  Coordinate  \
0        T1-MS                   NaN          A>A/G   1    14804874   
1        T1-MS                  SDHB          G>T/T   1    17380497   
2        T1-MS                 MUTYH          C>C/G   1    45797505   
3        T1-MS                 MUTYH          C>C/G   1    45797505   
4        T1-MS                   NaN          C>C/T   1   150860471   
5        T1-MS                 NR5A2          A>A/G   1   200007432   
6        T1-MS                   NaN          A>A/G   1   222164948   
7        T1-MS                   NaN          G>G/T   2    15782471   
8        T1-MS                   ALK          G>G/C   2    29416366   
9        T1-MS                   ALK          T>T/C   2    29416481   
10       T1-MS                   ALK          T>C/C   2    29416572   
11       T1-MS                   ALK          G>G/T   2    29445458   
12       T1-MS                   ALK          G>G/A   2    29449820   
13    